#### **Import Libs**

In [ ]:
import pandas as pd
import numpy as np

from keras import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD, Adam

import optuna

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import joblib

#### **Getting training && validation data**

In [ ]:
df_train = pd.read_csv('../databases/training.csv')
df_valid = pd.read_csv('../databases/validation.csv')

In [ ]:
X_train, y_train = df_train.drop(['IND_BOM_1_1'], axis=1), df_train['IND_BOM_1_1']
X_val, y_val = df_valid.drop(['IND_BOM_1_1'], axis=1), df_valid['IND_BOM_1_1']

In [ ]:
X_train = np.array(X_train)
X_val = np.array(X_val)

#### **Parameters selection**

In [ ]:
def objective(trial):
    model = DecisionTreeClassifier(
        max_depth=trial.suggest_int('max_depth', 2, 10),
        min_samples_split=trial.suggest_int('min_samples_split', 2, 10),
        min_samples_leaf=trial.suggest_int('min_samples_leaf', 1, 5),
        criterion=trial.suggest_categorical('criterion', ['gini', 'entropy']),
        splitter=trial.suggest_categorical('splitter', ['best', 'random'])
    )

    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)

    accuracy = accuracy_score(y_val, y_pred)

    return 1 - accuracy

In [ ]:
n_trials = 150

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=n_trials)

#### **Saving study**

In [ ]:
save_path = './optuna_studies/mlp_study.pkl'

joblib.dump(study, save_path)